In [40]:
fourXYZ = 'GGACTGGCCCTTAAGTTAGGCATCAGTGCCGACGCCGCTTATAAACTTGCTAAAGCAGCCTACTATTCAGGAACTACCGTTGAAGAAGCCTACAAATTGGCTCTTAAGTTGGGAATATCCGCTGATGCGGCTTATAAGCTTGCGGAAGCAGCCTACTATAGTGGAACCACTGTGGAAGAGGCCTATAAACTTGCATTAAAACTGGGGATCTCCGCTGATGCTGCATACAAATTAGCAAAAGCGGCGTACTACTCGGGCACTACCGTTGAAGAAGCATATAAGTTAGCACTGAAACTCGGGATCAGCGCGGACGCGGCTTACAAACTTGCAAAGGCGGCCTACTACTCGGGAACGACAGTGGAAGAGGCATATAAGCTTGCACTTAAACTGGGCATTAGTGCCGACGCAGCGTACAAGTTAGCCAAAGCCGCATATTATTCAGGTACGACTGTTGAAGAGGCATACAAATTAGCACTGAAGCTCGGCATATCCGCGGACGCCGCGTATAAACTGGCTAAAGCTGCTTACTATAGCGGCACGACAGTAGAGGAAGCTTACAAATTGGCCCTCAAGTTAGGTATCTCGGCGGATGCCGCGTATAAACTTGCAAAAGCGGCGTATTACTCAGGCACAACCGTTGAGGAAGCTTATAAATTGGCATTGAAATTAGGCATAAGTGCCGACGCTGCTTACAAACTGGCCAAGGCCGCGTACTACTCGGGAACAACGGTCGAAGAGGCATACAAGCTGGCATTAAAGTTAGGCATTAGCGCGGATGCTGCTTATAAATTGGCCAAAGCAGCATACTATTCTGGGACAACTGTCGAGGAAGCGTACAAGCTGGCACTGAAGTTGGGC'
eightE2Q = 'ATGAGCGAAGAACGCGTGGAAGAAGAACTGCGCCGCCTGGTGGAAGAAGAAGCGGAAGCGCGCGGCATTAGCCGCCGCGAAACCCTGGAACGCCTGAGCCTGCTGCTGTTTAGCATGCAGCTGGAAAAACTGGTGAAAGAAGAAGCGGAAGCGCGCGGCGTGAGCGTGGAAACCATTCGCGAAGAACTGGAACGCGAAGTGGATGAACGCCTGCGCGAACTGCGCGAAGAAGGCATTAGCCGCCGCGAAACCCTGGAACGCCTGAGCCTGCTGCTGTTTAGCATGCAGCTGGAAAAACTGGTGAAAGAAGAAGCGGAAGCGCGCGGCGTGAGCGTGGAAACCATTCGCGAAGAACTGGAACGCGAAGTGGATGAACGCCTGCGCGAACTGCGCGAAGAAGGCATTAGCCGCCGCGAAACCCTGGAACGCCTGAGCCTGCTGCTGTTTAGCATGCAGCTGGAAAAACTGGTGAAAGAAGAAGCGGAAGCGCGCGGCGTGAGCGTGGAAGAAATTCGCGAAGAACTGCGCCGCGAAGTGGATGAACGCCTGCGCGAACTGCGCGAAGAAGGCATTAGCCGCCGCGAAACCCTGGAACGCCTGAGCCTGCTGCTGTTTAGCATGCAGCTGGAAAAACTGGTGAAAGAAGAAGCGGAAGCGCGCGGCGTGAGCGTGGAAGAAATTCGCGAAGAACTGCGCCGCGAAGTGGATGAACGCCTGCGCGAACTGCGCGAAGAAGGCATTAGCCGCCGCGAAACCCTGGAACGCCTGAGCCTGCTGCTGTTTAGCATGCAGCTGGAAAAACTGGTGAAAGAAGAAGCGGAAGCGCGCGGCGTGAGCGTGGAAGAAATTCGCGAAGAACTGCGCCGCGAAGTGGAAGAACGCCTGCGCGAACTGCGCGAAGAAGGCGAA'
sevenSQ3 = 'GGCAGCAGCATGGGTTCAGACGAGCAACGACGTGAGCTTGAGGAGAAGATAAAGTTCAAGCTCGCTGAGTTGGCTTCAAAGTCAGAGGAAGAGCGGAAGGAGATCAAGCTCAGAGTTATAGCCTACGTTTTAGTTCAATTAGAGGACTTACAAAAGTTGTTATCTGACGAGCAACGTCGGGAGTTAGAGGAGAAGATAAAGTTCAAGCTTGCTGAGCTCGCCTCAAAGTCTGAGGAAGAGCGTAAGGAAATCAAGCTCAGAGTAATAGCATACGTTTTAGTTCAACTTGAGGACTTACAAAAGCTTTTATCTGACGAGCAACGTCGTGAGTTGGAAGAGAAGATAAAGTTCAAGTTAGCAGAGCTTGCTAGTAAGTCAGAAGAGGAGAGAAAGGAGATCAAGTTACGGGTTATAGCATACGTCTTGGTCCAACTTGAGGACTTACAAAAGTTACTCTCC'

In [68]:
from PIL import Image
import numpy as np

def bend_to_circle(img, inner_radius=200, outer_radius=600):
    """
    Warp a horizontal image into a ring shape using polar transform.
    - inner_radius: radius of the hole in the middle
    - outer_radius: outer radius of the ring
    """
    img = img.convert("RGBA")
    w, h = img.size

    # Output image size
    out_r = outer_radius
    out_size = out_r * 2
    out = Image.new("RGBA", (out_size, out_size), (255, 255, 255, 0))

    # Convert to numpy for speed
    src = np.array(img)
    dst = np.zeros((out_size, out_size, 4), dtype=np.uint8)

    center = out_size // 2
    ring_thickness = outer_radius - inner_radius

    for y in range(out_size):
        for x in range(out_size):
            dx = x - center
            dy = y - center
            r = np.sqrt(dx*dx + dy*dy)
            if inner_radius <= r < outer_radius:
                theta = np.arctan2(dy, dx)
                if theta < 0:
                    theta += 2 * np.pi
                # Map radius → vertical pixel, angle → horizontal pixel
                src_x = int(theta / (2 * np.pi) * w)
                src_y = int((r - inner_radius) / ring_thickness * h)
                if 0 <= src_x < w and 0 <= src_y < h:
                    dst[y, x] = src[src_y, src_x]

    return Image.fromarray(dst, "RGBA")


def create_concentric_rings(img_path, innermost_radius=100, ring_height=50, margin=10, target_rings=None):
    """
    Create concentric rings from a long horizontal image with proper area scaling.
    
    Parameters:
    - img_path: path to the long horizontal image
    - innermost_radius: radius of the innermost ring's inner edge
    - ring_height: height of each ring (thickness)
    - margin: spacing between rings
    - target_rings: optional, number of rings to create (will distribute image evenly)
    
    Returns:
    - composite image with all concentric rings
    """
    # Load the long horizontal image
    long_img = Image.open(img_path).convert("RGBA")
    img_width, img_height = long_img.size
    
    total_source_area = img_width * img_height
    
    # If target_rings specified, calculate how to distribute the image
    if target_rings:
        rings_config = []
        current_inner = innermost_radius
        
        # Calculate total area needed across all rings
        total_ring_area = 0
        for i in range(target_rings):
            inner_r = current_inner
            outer_r = inner_r + ring_height
            ring_area = np.pi * (outer_r**2 - inner_r**2)
            total_ring_area += ring_area
            current_inner = outer_r + margin
        
        # Distribute source image proportional to ring area
        current_x = 0
        current_inner = innermost_radius
        
        for i in range(target_rings):
            inner_r = current_inner
            outer_r = inner_r + ring_height
            ring_area = np.pi * (outer_r**2 - inner_r**2)
            
            # Width proportional to this ring's area
            strip_width = int((ring_area / total_ring_area) * img_width)
            
            # Ensure we don't exceed image bounds
            if current_x + strip_width > img_width:
                strip_width = img_width - current_x
            
            rings_config.append({
                'inner_r': inner_r,
                'outer_r': outer_r,
                'start_x': current_x,
                'width': strip_width
            })
            
            current_x += strip_width
            current_inner = outer_r + margin
    else:
        # Auto-calculate rings to use entire image
        rings_config = []
        current_x = 0
        current_inner = innermost_radius
        remaining_area = total_source_area
        
        while current_x < img_width:
            inner_r = current_inner
            outer_r = inner_r + ring_height
            
            # Area of this ring
            ring_area = np.pi * (outer_r**2 - inner_r**2)
            
            # Width of source needed: ring_area = strip_width * img_height
            strip_width = int(ring_area / img_height)
            
            # Check bounds
            if current_x + strip_width > img_width:
                strip_width = img_width - current_x
                if strip_width < 10:
                    break
            
            rings_config.append({
                'inner_r': inner_r,
                'outer_r': outer_r,
                'start_x': current_x,
                'width': strip_width
            })
            
            current_x += strip_width
            current_inner = outer_r + margin
    
    # Create rings
    rings = []
    for config in rings_config:
        if config['width'] <= 0:
            continue
            
        # Extract strip
        strip = long_img.crop((
            config['start_x'],
            0,
            config['start_x'] + config['width'],
            img_height
        ))
        
        # Bend to circle
        ring = bend_to_circle(strip, inner_radius=config['inner_r'], outer_radius=config['outer_r'])
        rings.append(ring)
    
    # Composite all rings
    if not rings:
        return None
    
    max_size = max(ring.size[0] for ring in rings)
    final_img = Image.new("RGBA", (max_size, max_size), (255, 255, 255, 0))
    
    for ring in rings:
        offset = (max_size - ring.size[0]) // 2
        final_img.paste(ring, (offset, offset), ring)
    
    return final_img


# Example usage
if __name__ == "__main__":
    # Auto mode - uses entire image
    result = create_concentric_rings(
        "test_dna_font.png",
        innermost_radius=500,
        ring_height=500,
        margin=15
    )
    
    if result:
        result.save("dna_concentric.png")
        print("Saved dna_concentric.png")
    else:
        print("No rings created")

Saved dna_concentric.png


In [ ]:
from PIL import Image
import numpy as np

def bend_to_circle(img, inner_radius=200, outer_radius=600):
    """
    Warp a horizontal image into a ring shape using polar transform.
    - inner_radius: radius of the hole in the middle
    - outer_radius: outer radius of the ring
    """
    img = img.convert("RGBA")
    w, h = img.size

    # Output image size
    out_r = outer_radius
    out_size = out_r * 2
    out = Image.new("RGBA", (out_size, out_size), (255, 255, 255, 0))

    # Convert to numpy for speed
    src = np.array(img)
    dst = np.zeros((out_size, out_size, 4), dtype=np.uint8)

    center = out_size // 2
    ring_thickness = outer_radius - inner_radius

    for y in range(out_size):
        for x in range(out_size):
            dx = x - center
            dy = y - center
            r = np.sqrt(dx*dx + dy*dy)
            if inner_radius <= r < outer_radius:
                theta = np.arctan2(dy, dx)
                if theta < 0:
                    theta += 2 * np.pi
                # Map radius → vertical pixel, angle → horizontal pixel
                src_x = int(theta / (2 * np.pi) * w)
                src_y = int((r - inner_radius) / ring_thickness * h)
                if 0 <= src_x < w and 0 <= src_y < h:
                    dst[y, x] = src[src_y, src_x]

    return Image.fromarray(dst, "RGBA")


def create_concentric_rings(img_path, innermost_radius=100, ring_height=50, margin=10):
    """
    Create concentric rings from a long horizontal image.
    
    Parameters:
    - img_path: path to the long horizontal image
    - innermost_radius: radius of the innermost ring's inner edge
    - ring_height: height of each ring (thickness)
    - margin: spacing between rings
    
    Returns:
    - composite image with all concentric rings
    """
    # Load the long horizontal image
    long_img = Image.open(img_path).convert("RGBA")
    img_width, img_height = long_img.size
    
    # Calculate how many rings we can make
    rings = []
    current_x = 0
    current_inner_radius = innermost_radius
    
    while current_x < img_width:
        inner_r = current_inner_radius
        outer_r = inner_r + ring_height
        
        # Calculate circumference at the middle of this ring
        mid_radius = (inner_r + outer_r) / 2
        circumference = 2 * np.pi * mid_radius
        
        # Width of strip needed for this ring
        strip_width = int(circumference)
        
        # Check if we have enough image left
        if current_x + strip_width > img_width:
            strip_width = img_width - current_x
            if strip_width < 10:  # Skip if too small
                break
        
        # Extract the strip
        strip = long_img.crop((current_x, 0, current_x + strip_width, img_height))
        
        # Bend to circle
        ring = bend_to_circle(strip, inner_radius=inner_r, outer_radius=outer_r)
        rings.append(ring)
        
        # Move to next ring
        current_x += strip_width
        current_inner_radius = outer_r + margin
    
    # Composite all rings onto one image
    if not rings:
        return None
    
    # Find the size needed (based on outermost ring)
    max_size = max(ring.size[0] for ring in rings)
    final_img = Image.new("RGBA", (max_size, max_size), (255, 255, 255, 0))
    
    # Paste each ring centered
    for ring in rings:
        offset = (max_size - ring.size[0]) // 2
        final_img.paste(ring, (offset, offset), ring)
    
    return final_img




Saved dna_concentric.png


In [70]:
# Example usage
result = create_concentric_rings(
    "test_dna_font.png",
    innermost_radius=1500,
    ring_height=500,
    margin=10
)

if result:
    result.save("dna_concentric.png")
    print("Saved dna_concentric.png")
else:
    print("No rings created - image may be too short")


Saved dna_concentric.png


In [69]:
# Example usage
result = create_concentric_rings(
    "7SQ3_dna_font.png",
    innermost_radius=300,
    ring_height=700,
    margin=10
)

if result:
    result.save("7SQ3_dna_font_concentric.png")
    print("Saved 7SQ3_dna_font_concentric.png")
else:
    print("No rings created - image may be too short")



Saved 7SQ3_dna_font_concentric.png
